<a href="https://colab.research.google.com/github/ddekun/NLP/blob/lesson8/lesson8/hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в обработку естественного языка

### Урок 8. Рекуррентные нейронные сети RNN LSTM GRU

In [25]:
!pip install stop-words
!pip install pymorphy2

In [26]:
import numpy as np
import pandas as pd

import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, SpatialDropout1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
from tensorflow.keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import matplotlib.pyplot as plt

In [27]:
!pip install xlrd

In [28]:
df = pd.read_excel('/content/drive/MyDrive/Geekbrains/Introduction to natural language processing/Lesson8/отзывы за лето.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [29]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [30]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [31]:
df_train.head()

,Rating,Content,Date
0,5,супер,2017-07-29
1,5,отличный приложение удобно управлять счёт,2017-06-25
2,5,около,2017-08-03
3,5,0 минус,2017-08-02
4,5,приятный положение,2017-07-31


In [32]:
text_corpus_train = df_train['Content'].values
text_corpus_test = df_test['Content'].values

In [33]:
tokenizer = Tokenizer(num_words=None,
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False,
                      split = ' ')

tokenizer.fit_on_texts(text_corpus_train)

In [34]:
sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

In [35]:
word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [36]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating'])
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [37]:
num_classes = 5

y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)

y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [38]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              output_dim=128,
              input_length=training_length))
model.add(Conv1D(128, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [39]:
early_stopping=EarlyStopping(monitor='val_loss')

epochs = 20
batch_size = 512

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
26/26 [==============================] - 13s 479ms/step - loss: 1.0939 - accuracy: 0.6643 - val_loss: 0.8097 - val_accuracy: 0.7049
Epoch 2/20
26/26 [==============================] - 12s 473ms/step - loss: 0.7227 - accuracy: 0.7519 - val_loss: 0.7189 - val_accuracy: 0.7436
Epoch 3/20
26/26 [==============================] - 12s 447ms/step - loss: 0.6296 - accuracy: 0.7782 - val_loss: 0.6975 - val_accuracy: 0.7457
Epoch 4/20
26/26 [==============================] - 10s 379ms/step - loss: 0.5672 - accuracy: 0.7916 - val_loss: 0.6879 - val_accuracy: 0.7547
Epoch 5/20
26/26 [==============================] - 12s 463ms/step - loss: 0.5066 - accuracy: 0.8152 - val_loss: 0.6965 - val_accuracy: 0.7574


In [40]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 104ms/step - loss: 0.6756 - accuracy: 0.7690


Test loss: 0.6755661964416504
Test accuracy: 0.7689577341079712


**RNN**

In [41]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
26/26 [==============================] - 10s 291ms/step - loss: 1.0771 - accuracy: 0.6834 - val_loss: 0.9037 - val_accuracy: 0.7049
Epoch 2/20
26/26 [==============================] - 5s 192ms/step - loss: 0.9429 - accuracy: 0.7055 - val_loss: 0.8339 - val_accuracy: 0.7091
Epoch 3/20
26/26 [==============================] - 6s 213ms/step - loss: 0.8040 - accuracy: 0.7314 - val_loss: 0.7715 - val_accuracy: 0.7270
Epoch 4/20
26/26 [==============================] - 7s 250ms/step - loss: 0.7351 - accuracy: 0.7592 - val_loss: 0.7353 - val_accuracy: 0.7388
Epoch 5/20
26/26 [==============================] - 5s 193ms/step - loss: 0.6562 - accuracy: 0.7809 - val_loss: 0.7133 - val_accuracy: 0.7408
Epoch 6/20
26/26 [==============================] - 7s 281ms/step - loss: 0.5879 - accuracy: 0.7950 - val_loss: 0.7010 - val_accuracy: 0.7519
Epoch 7/20
26/26 [==============================] - 5s 188ms/step - loss: 0.5266 - accuracy: 0.8150 - val_loss: 0.7212 - val_accuracy: 0.7478


In [43]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 63ms/step - loss: 0.7137 - accuracy: 0.7581


Test loss: 0.7136932611465454
Test accuracy: 0.7581477761268616


In [44]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [45]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
26/26 [==============================] - 27s 884ms/step - loss: 1.2262 - accuracy: 0.6958 - val_loss: 0.9294 - val_accuracy: 0.7049
Epoch 2/20
26/26 [==============================] - 22s 843ms/step - loss: 0.8822 - accuracy: 0.7060 - val_loss: 0.7912 - val_accuracy: 0.7111
Epoch 3/20
26/26 [==============================] - 23s 846ms/step - loss: 0.7640 - accuracy: 0.7454 - val_loss: 0.7293 - val_accuracy: 0.7436
Epoch 4/20
26/26 [==============================] - 34s 1s/step - loss: 0.6914 - accuracy: 0.7664 - val_loss: 0.7102 - val_accuracy: 0.7436
Epoch 5/20
26/26 [==============================] - 27s 1s/step - loss: 0.6396 - accuracy: 0.7776 - val_loss: 0.6940 - val_accuracy: 0.7547
Epoch 6/20
26/26 [==============================] - 22s 867ms/step - loss: 0.5940 - accuracy: 0.7892 - val_loss: 0.6951 - val_accuracy: 0.7567


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64,recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

**CNN -> RNN**

In [ ]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

**RNN -> CNN**

In [ ]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(64, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])